In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copy

ERROR! Session/line number was not unique in database. History logging moved to new session 70


In [ ]:
local_zip = 'D:\\Downloads\\kagglecatsanddogs_3367a.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('D:\\Other\\datasets')
zip_ref.close()

In [3]:
ds_loc = 'D:\\Other\\datasets'
print(len(os.listdir(ds_loc+'\\PetImages\\Cat\\')))
print(len(os.listdir(ds_loc+'\\PetImages\\Dog\\')))

12501
12501


In [ ]:
os.mkdir(ds_loc+"\\PetImages\\train")
os.mkdir(ds_loc+"\\PetImages\\test")

for class_name in ['cats','dogs']:
    os.mkdir(ds_loc+"\\PetImages\\train\\"+class_name)
    os.mkdir(ds_loc+"\\PetImages\\test\\"+class_name)

In [ ]:
from random import shuffle


def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    im_names = [i for i in os.listdir(SOURCE) if i[-1]=='g'\
                and os.path.getsize(SOURCE+"\\"+i)!=0]
    shuffle(im_names)

    for im in im_names[:int(len(im_names)*SPLIT_SIZE)]:
        copy(SOURCE+"\\"+im, TRAINING+"\\"+im)

    for im in im_names[int(len(im_names)*SPLIT_SIZE):]:
        copy(SOURCE+"\\"+im, TESTING+"\\"+im)



CAT_SOURCE = ds_loc + "\\PetImages\\Cat"
TRAINING_CATS = ds_loc + "\\PetImages\\train\\cats"
TESTING_CATS = ds_loc + "\\PetImages\\test\\cats"
DOG_SOURCE = ds_loc + "\\PetImages\\Dog"
TRAINING_DOGS = ds_loc + "\\PetImages\\train\\dogs"
TESTING_DOGS = ds_loc + "\\PetImages\\test\\dogs"

split_size = 0.9
split_data(CAT_SOURCE, TRAINING_CATS, TESTING_CATS, split_size)
split_data(DOG_SOURCE, TRAINING_DOGS, TESTING_DOGS, split_size)

In [4]:
CAT_SOURCE = ds_loc + "\\PetImages\\Cat"
TRAINING_CATS = ds_loc + "\\PetImages\\train\\cats"
TESTING_CATS = ds_loc + "\\PetImages\\test\\cats"
DOG_SOURCE = ds_loc + "\\PetImages\\Dog"
TRAINING_DOGS = ds_loc + "\\PetImages\\train\\dogs"
TESTING_DOGS = ds_loc + "\\PetImages\\test\\dogs"

print(len(os.listdir(TRAINING_CATS)))
print(len(os.listdir(TESTING_CATS)))
print(len(os.listdir(TRAINING_DOGS)))
print(len(os.listdir(TESTING_DOGS)))

11249
1250
11249
1250


In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), input_shape=(150,150,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

In [6]:
TRAINING_DIR = ds_loc+"\\PetImages\\train"
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    target_size=(150,150),
                                                    class_mode='binary')

TESTING_DIR = ds_loc+"\\PetImages\\test"
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(TESTING_DIR,
                                                    target_size=(150,150),
                                                    batch_size=2,
                                                    class_mode='binary')

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [7]:
history = model.fit(train_generator, epochs=5,
                    verbose=1, validation_data=test_generator)



Epoch 1/5
 61/704 [=>............................] - ETA: 13:11 - loss: 0.9215 - accuracy: 0.5282  

KeyboardInterrupt: 